read json

In [ ]:
bq load --source_format=NEWLINE_DELIMITED_JSON --autodetect OneWaySolution.yelp-users-json gs://owshq-zone/files/users/yelp_academic_dataset_user_2021.json

### Cluster table
- Automatic recluster

In [ ]:
create table `silver-charmer-243611.OneWaySolution.clustered-yelp-reviews`
(
    date TIMESTAMP,
    useful INTEGER,
)
CLUSTER BY date AS 
(
    SELECT date, useful from `silver-charmer-243611.OneWaySolution.clustered-yelp-reviews-json`
)

Snapshot table

In [ ]:
CREATE SNAPSHOT TABLE `silver-charmer-243611.OneWaySolution.yelp-reviews-snapshot`
CLONE `silver-charmer-243611.OneWaySolution.clustered-yelp-reviews``
FOR SYSTEM_TIME AS OF TIMESTAMP_SUB(CURRENT_TIMESTAMP(),INTERVAL 1  HOUR);

Materialized View

In [ ]:
CREATE MATERIALIZED VIEW `silver-charmer-243611.OneWaySolution.m-view-yelp-reviews`
AS
(
    SELECT user.user_id,
    SUM(user.review_count) AS reviews,
    COUNT(reviews.stars) AS stars
FROM `silver-charmer-243611.OneWaySolution.yelp-users-json` AS users
INNER JOIN `silver-charmer-243611.OneWaySolution.clustered-yelp-reviews-json` as reviews
ON users.user_id = reviews.user_id
GROUP BY users.user_id
);

External table

In [ ]:
CREATE OR REPLACE EXTERNAL TABLE `silver-charmer-243611.OneWaySolution.ex-tb-golg-reviews`
OPTIONS
(
    format= 'PARQUET',
    uris = ['gs://svc-orion-dev/files/gold-reviews/*']
);